In [47]:
#Question 1 - probabilities

In [48]:
import numpy as np
import scipy.stats as stats

# Parameters
n_questions = 20
p_N = 11/20  # Probability of knowing the correct answer

# Define binomial PMF function
def binomial_pmf(k, n, p):
    return stats.binom.pmf(k, n, p)

# Compute P(N = k) for k = 0 to 20
N_values = np.arange(0, n_questions + 1)
P_N = binomial_pmf(N_values, n_questions, p_N)

# Compute P(Y >= T) and P(Y >= T | N < 10)
thresholds = np.arange(0, n_questions + 1)
probabilities = []

for T in thresholds:
    # Compute P(N < 10)
    P_N_less_10 = np.sum(P_N[:10])
    
    # Compute P(Y >= T | N < 10)
    P_Y_ge_T_given_N_less_10 = 0
    for N in range(10):  # Only consider N < 10
        P_N_k = P_N[N]  # P(N = k)
        P_Z = 1 - stats.binom.cdf(T - 1 - N, n_questions - N, 0.5)  # P(Z >= T - N)
        P_Y_ge_T_given_N_less_10 += P_N_k * P_Z
    
    # Compute P(Y >= T)
    P_Y_ge_T = 0
    for N in range(n_questions + 1):  # Sum over all N
        P_N_k = P_N[N]  # P(N = k)
        P_Z = 1 - stats.binom.cdf(T - 1 - N, n_questions - N, 0.5)  # P(Z >= T - N)
        P_Y_ge_T += P_N_k * P_Z
    
    # Compute conditional probability
    P_N_less_10_given_Y_ge_T = P_Y_ge_T_given_N_less_10 / P_Y_ge_T if P_Y_ge_T > 0 else 0
    probabilities.append(P_N_less_10_given_Y_ge_T)

# Answer for part 1

problem11_probabilities = probabilities
# Convert NumPy float64 to standard Python floats
problem11_probabilities = [float(p) for p in problem11_probabilities]

# Print the cleaned list
print(problem11_probabilities)




[0.2492893598284119, 0.2492893598283289, 0.2492893598226105, 0.24928935963549279, 0.24928935576839334, 0.2492892991583497, 0.24928867518930248, 0.24928330207958455, 0.24924628523366024, 0.2490390263029906, 0.24808569900431438, 0.24460820014975931, 0.2349439695781524, 0.21475641513175914, 0.18267139196620996, 0.14272522447072045, 0.10227042692681913, 0.06762809950564577, 0.04166472439122744, 0.024151134340423375, 0.013287462679601611]


In [49]:
# Solve Part 2: Find the smallest T such that P(N >= 10 | Y >= T) >= 0.90

# Compute P(N >= 10 | Y >= T) using the complement: P(N >= 10 | Y >= T) = 1 - P(N < 10 | Y >= T)
P_N_ge_10_given_Y_ge_T = [1 - p for p in problem11_probabilities]

# Find the smallest T where P(N >= 10 | Y >= T) >= 0.90
problem12_T = None
for T, prob in enumerate(P_N_ge_10_given_Y_ge_T):
    if prob >= 0.90:
        problem12_T = T
        break  # Stop at the first valid T

# Display the result
print(f"The smallest T such that P(N >= 10 | Y >= T) >= 0.90 is: {problem12_T}")


The smallest T such that P(N >= 10 | Y >= T) >= 0.90 is: 17


In [50]:
#Question 2-Random Variable generation and transformation (already done in assignment 2)

In [51]:
def problem4_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    

    # Parameters for the LCG (these satisfy the Hull-Dobell theorem)
    M = 2**31 - 1  # large modulus
    a = 1103515245  # multiplier
    b = 12345       # increment
    
    # Initializing the seed
    u = seed
    out = []
    for _ in range(size):
        u = (a * u + b) % M
        out.append(u)
    return out

In [52]:
def problem2_uniform(generator=None, period=1, size=None, seed=0):
    """
    Converts LCG output into uniform [0,1] distributed random numbers.

    Parameters:
    -----------
    generator : function
        A function that generates pseudo-random numbers (e.g., problem2_LCG).
    period : int
        The period of the generator.
    size : int
        The number of uniform samples required.
    seed : int
        The seed value for reproducibility.

    Returns:
    --------
    out : list
        A list of random numbers in the range [0,1].
    """
    # Get pseudo-random numbers from the LCG
    random_numbers = generator(size=size, seed=seed)
    
    # Normalize to get uniform [0,1] values
    M = 2**32  # Same modulus as in LCG
    uniform_numbers = [x / M for x in random_numbers]
    
    return uniform_numbers



In [53]:
import math

def problem2_accept_reject(uniformGenerator=None, size=None, seed=0):
    """
    Generates samples using the Accept-Reject method with target distribution:
        p_0(x) = (pi/2) * |sin(2πx)|
    """
    if uniformGenerator is None:
        raise ValueError("A valid uniform generator function must be provided.")

    out = []  
    c = math.pi / 2  

    current_seed = seed

    while len(out) < size:
        # Generate U ~ Uniform(0,1) and V ~ Uniform(0,1) using the provided generator
        u = uniformGenerator(generator=problem2_LCG, size=1, seed=current_seed)[0]
        v = uniformGenerator(generator=problem2_LCG, size=1, seed=current_seed + 1)[0]

        # Increment the seed to avoid repetition
        current_seed += 2

        # Acceptance condition
        if v <= (math.pi / 2) * abs(math.sin(2 * math.pi * u)) / c:
            out.append(u)

    return out




In [54]:
#this just shows how it would be implemented fully sequrentially
import math

def problem2_LCG(size=None, seed=0):
    """
    A Linear Congruential Generator (LCG) that generates pseudo-random numbers.
    """
    # LCG Parameters (Hull-Dobell theorem satisfied)
    a = 1664525
    c = 1013904223
    M = 2**32  

    X = seed
    random_numbers = []
    
    for _ in range(size):
        X = (a * X + c) % M
        random_numbers.append(X)
    
    return random_numbers

def problem2_uniform(generator=None, size=None, seed=0):
    """
    Converts LCG output into uniform [0,1] distributed random numbers.
    """
    if generator is None:
        raise ValueError("Generator function cannot be None.")

    # Get pseudo-random numbers from LCG
    random_numbers = generator(size=size, seed=seed)
    
    # Normalize to [0,1]
    M = 2**32  
    uniform_numbers = [x / M for x in random_numbers]
    
    return uniform_numbers

def problem2_accept_reject(uniformGenerator=None, size=None, seed=0):
    """
    Generates samples using the Accept-Reject method with target distribution:
        p_0(x) = (pi/2) * |sin(2πx)|
    """
    if uniformGenerator is None:
        raise ValueError("A valid uniform generator function must be provided.")

    out = []  
    c = math.pi / 2  

    current_seed = seed

    while len(out) < size:
        # Generate U ~ Uniform(0,1) and V ~ Uniform(0,1) using the provided generator
        u = uniformGenerator(generator=problem2_LCG, size=1, seed=current_seed)[0]
        v = uniformGenerator(generator=problem2_LCG, size=1, seed=current_seed + 1)[0]

        # Increment the seed to avoid repetition
        current_seed += 2

        # Acceptance condition
        if v <= (math.pi / 2) * abs(math.sin(2 * math.pi * u)) / c:
            out.append(u)

    return out

# Example Usage
size = 10
seed = 42

# Generate samples using the uniform generator
samples = problem2_accept_reject(uniformGenerator=problem2_uniform, size=size, seed=seed)

# Print results
print("Generated Samples:", samples)


Generated Samples: [0.2523451747838408, 0.2531202796380967, 0.2538953844923526, 0.2546704893466085, 0.25544559420086443, 0.25622069905512035, 0.25699580390937626, 0.2577709087636322, 0.2585460136178881, 0.259321118472144]


In [55]:
#Problem 3 -Concentration measure

In [56]:
#have been done in 1st assignment

In [57]:
#problem 4 -spam detection with random variables

In [58]:
def estimate_spam_probability(sms_data):
    """
    Computes P(Y=1 | "free" or "prize" in X).
    
    Parameters:
    -----------
    sms_data : list of tuples
        Each tuple contains (SMS text, label) where label is 0 (not spam) or 1 (spam).
    
    Returns:
    --------
    float
        Estimated probability of spam given that the SMS contains "free" or "prize".
    """
    
    # Convert words to lowercase for case-insensitive matching
    keyword_filter = lambda text: "free" in text.lower() or "prize" in text.lower()
    
    # Filter messages containing "free" or "prize"
    filtered_messages = [(text, label) for text, label in sms_data if keyword_filter(text)]
    
    if len(filtered_messages) == 0:
        return 0  # Avoid division by zero if no messages contain the words

    # Count spam messages among the filtered ones
    spam_count = sum(1 for _, label in filtered_messages if label == 1)
    
    # Compute probability
    probability = spam_count / len(filtered_messages)
    
    return probability

# Example Usage
# Assume sms_no_spam is a list of tuples [(text1, label1), (text2, label2), ...]
spam_probability = estimate_spam_probability(sms_no_spam)

# Print result
print(f"P(Y = 1 | 'free' or 'prize' in X) ≈ {spam_probability:.4f}")


NameError: name 'sms_no_spam' is not defined

In [ ]:
import numpy as np

def compute_hoeffding_confidence_interval(estimated_prob, n_samples, confidence=0.90):
    """
    Computes the confidence interval for a probability estimate using Hoeffding's inequality.

    Parameters:
    -----------
    estimated_prob : float
        The empirical probability estimate (P̂).
    n_samples : int
        The number of samples used to compute the probability estimate.
    confidence : float
        The confidence level (default is 90%).

    Returns:
    --------
    tuple (lower_bound, upper_bound)
        The confidence interval for the estimated probability.
    """
    delta = 1 - confidence  # Confidence level (e.g., 0.10 for 90% interval)
    epsilon = np.sqrt(np.log(2 / delta) / (2 * n_samples))  # Compute Hoeffding bound

    # Compute confidence interval
    lower_bound = max(0, estimated_prob - epsilon)  # Ensure non-negative
    upper_bound = min(1, estimated_prob + epsilon)  # Ensure probability stays ≤ 1
    
    return lower_bound, upper_bound

# Example Usage
n_samples = 100  # Replace with actual count of SMS messages used
estimated_prob = 0.76  # Replace with actual probability from Part 1

# Compute confidence interval
confidence_interval = compute_hoeffding_confidence_interval(estimated_prob, n_samples)

# Print results
print(f"Estimated Probability (P̂): {estimated_prob:.4f}")
print(f"90% Confidence Interval: {confidence_interval}")

# Assertions to ensure correct output
assert isinstance(confidence_interval, tuple)
assert len(confidence_interval) == 2


In [ ]:
import numpy as np

def estimate_spam_probability_for_free_twice(sms_data):
    """
    Computes P(Y=1 | "free" appears twice in X).
    
    Parameters:
    -----------
    sms_data : list of tuples
        Each tuple contains (SMS text, label) where label is 0 (not spam) or 1 (spam).
    
    Returns:
    --------
    float
        Estimated probability of spam given "free" appears twice in the SMS.
    int
        Number of samples where "free" appears twice.
    """
    
    # Filter messages where "free" appears at least twice (case insensitive)
    keyword_filter = lambda text: text.lower().count("free") >= 2
    filtered_messages = [(text, label) for text, label in sms_data if keyword_filter(text)]
    
    if len(filtered_messages) == 0:
        return 0, 0  # Avoid division by zero if no messages match the condition

    # Count spam messages among the filtered ones
    spam_count = sum(1 for _, label in filtered_messages if label == 1)
    
    # Compute probability
    probability = spam_count / len(filtered_messages)
    
    return probability, len(filtered_messages)

def compute_hoeffding_confidence_interval(estimated_prob, n_samples, confidence=0.90):
    """
    Computes the confidence interval for a probability estimate using Hoeffding's inequality.

    Parameters:
    -----------
    estimated_prob : float
        The empirical probability estimate (P̂).
    n_samples : int
        The number of samples used to compute the probability estimate.
    confidence : float
        The confidence level (default is 90%).

    Returns:
    --------
    tuple (lower_bound, upper_bound)
        The confidence interval for the estimated probability.
    """
    delta = 1 - confidence  # Confidence level (e.g., 0.10 for 90% interval)
    if n_samples == 0:
        return (0, 1)  # If no samples found, return full range [0,1]
    
    epsilon = np.sqrt(np.log(2 / delta) / (2 * n_samples))  # Compute Hoeffding bound

    # Compute confidence interval
    lower_bound = max(0, estimated_prob - epsilon)  # Ensure non-negative
    upper_bound = min(1, estimated_prob + epsilon)  # Ensure probability stays ≤ 1
    
    return lower_bound, upper_bound

# Example Usage
# Assume sms_no_spam is a list of tuples [(text1, label1), (text2, label2), ...]
estimated_prob, n_samples = estimate_spam_probability_for_free_twice(sms_no_spam)

# Compute confidence interval
confidence_interval = compute_hoeffding_confidence_interval(estimated_prob, n_samples)

# Print results
print(f"P(Y = 1 | 'free' appears twice) ≈ {estimated_prob:.4f}")
print(f"Number of relevant SMS samples: {n_samples}")
print(f"90% Confidence Interval: {confidence_interval}")


In [ ]:
#problem 5 markov chains

In [ ]:
import pandas as pd

# Load the dataset
file_path = "data/flights.csv"  # Ensure this path is correct
df = pd.read_csv(file_path)

# Extract relevant information
number_of_cities = df['origin'].nunique()  # Count unique cities in the dataset
number_of_userCodes = df['user_id'].nunique()  # Count unique users (travelers)
number_of_observations = len(df)  # Total number of flights (observations)

# Print results
print(f"Number of cities: {number_of_cities}")
print(f"Number of unique users: {number_of_userCodes}")
print(f"Number of flight observations: {number_of_observations}")


In [ ]:
import numpy as np

# Step 1: Enumerate cities in alphabetical order
unique_cities = sorted(df['origin'].unique())  # Get unique city names in sorted order
city_to_index = {city: idx for idx, city in enumerate(unique_cities)}

# Step 2: Map cities to indices in the dataset
df['origin_index'] = df['origin'].map(city_to_index)
df['destination_index'] = df['destination'].map(city_to_index)

# Step 3: Create transition frequency dictionary
transition_pairs = list(zip(df['origin_index'], df['destination_index']))
transition_counts = makeFreqDict(transition_pairs)  # Using the provided function

# Step 4: Build the transition matrix
num_cities = len(unique_cities)
transition_matrix = np.zeros((num_cities, num_cities))

# Fill the matrix with frequency counts
for (from_city, to_city), count in transition_counts.items():
    transition_matrix[from_city, to_city] = count

# Step 5: Normalize the transition matrix (convert counts to probabilities)
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

# Display the transition matrix
import ace_tools as tools
import pandas as pd

transition_df = pd.DataFrame(transition_matrix, index=unique_cities, columns=unique_cities)
tools.display_dataframe_to_user(name="Transition Matrix", dataframe=transition_df)

# Print confirmation
print("Markov chain transition matrix computed successfully!")


In [ ]:
import numpy as np

def compute_stationary_distribution(transition_matrix):
    """
    Computes the stationary distribution of a Markov chain given its transition matrix.

    Parameters:
    -----------
    transition_matrix : numpy array (n_cities x n_cities)
        The Markov transition matrix.

    Returns:
    --------
    numpy array
        The stationary distribution vector.
    """
    # Compute the left eigenvector corresponding to eigenvalue 1
    eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)

    # Find the eigenvector corresponding to eigenvalue closest to 1
    stationary_vector = np.real(eigenvectors[:, np.isclose(eigenvalues, 1)].flatten())

    # Normalize the vector to sum to 1 (probability distribution)
    stationary_distribution = stationary_vector / stationary_vector.sum()

    return stationary_distribution

# Compute stationary distribution using our transition matrix
stationary_distribution = compute_stationary_distribution(transition_matrix)

# Display the result
import ace_tools as tools
import pandas as pd

stationary_df = pd.DataFrame(stationary_distribution, index=unique_cities, columns=["Stationary Probability"])
tools.display_dataframe_to_user(name="Stationary Distribution", dataframe=stationary_df)

# Print confirmation
print("Stationary distribution computed successfully!")


In [ ]:
import numpy as np

def compute_return_probability(transition_matrix, start_city, steps, city_to_index):
    """
    Computes the probability of returning to a city after a given number of steps.

    Parameters:
    -----------
    transition_matrix : numpy array
        The Markov chain transition matrix.
    start_city : str
        The city name from which we start.
    steps : int
        The number of steps to take in the Markov process.
    city_to_index : dict
        A dictionary mapping city names to indices.

    Returns:
    --------
    float
        The probability of returning to the start city after 'steps' transitions.
    """
    # Compute P^steps (transition matrix raised to power 'steps')
    P_n = np.linalg.matrix_power(transition_matrix, steps)

    # Get the index of the starting city
    city_index = city_to_index[start_city]

    # Probability of returning to the same city after 'steps' steps
    return_probability = P_n[city_index, city_index]

    return return_probability

# Define start city and number of steps
start_city = "Aracaju (SE)"
steps = 3

# Compute return probability
return_probability = compute_return_probability(transition_matrix, start_city, steps, city_to_index)

# Print the result
print(f"Probability of returning to '{start_city}' after {steps} steps: {return_probability:.4f}")


In [ ]:
#problem 6 blackbox testing


In [ ]:
import numpy as np

def compute_precision(y_true, y_pred):
    """
    Computes precision for class 1.

    Parameters:
    y_true : list or numpy array
        True labels (0 = non-spam, 1 = spam).
    y_pred : list or numpy array
        Predicted labels.

    Returns:
    precision : float
        Precision for class 1.
    n_positive_preds : int
        Number of predicted positive samples (TP + FP).
    """
    TP = sum((y_pred == 1) & (y_true == 1))
    FP = sum((y_pred == 1) & (y_true == 0))
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    return precision, (TP + FP)

def compute_hoeffding_bound(n_samples, confidence=0.95):
    """
    Computes the Hoeffding bound for a given number of samples.

    Parameters:
    n_samples : int
        Number of samples used in the calculation.
    confidence : float
        Confidence level.

    Returns:
    l : float
        Hoeffding confidence bound.
    """
    delta = 1 - confidence
    l = np.sqrt(np.log(2 / delta) / (2 * n_samples)) if n_samples > 0 else 0
    return l

# Example Usage
y_true = np.array([...])  # Replace with actual test labels
y_pred = np.array([...])  # Replace with actual predicted labels

precision, n_positive_preds = compute_precision(y_true, y_pred)
l = compute_hoeffding_bound(n_positive_preds)

print(f"Precision: {precision:.4f}")
print(f"95% Confidence Interval: ({max(0, precision - l):.4f}, {min(1, precision + l):.4f})")


In [ ]:
def compute_recall(y_true, y_pred):
    """
    Computes recall for class 1.

    Parameters:
    y_true : list or numpy array
        True labels (0 = non-spam, 1 = spam).
    y_pred : list or numpy array
        Predicted labels.

    Returns:
    recall : float
        Recall for class 1.
    n_actual_positives : int
        Number of actual positive samples (TP + FN).
    """
    TP = sum((y_pred == 1) & (y_true == 1))
    FN = sum((y_pred == 0) & (y_true == 1))
    
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    return recall, (TP + FN)

# Example Usage
recall, n_actual_positives = compute_recall(y_true, y_pred)
l = compute_hoeffding_bound(n_actual_positives)

print(f"Recall: {recall:.4f}")
print(f"95% Confidence Interval: ({max(0, recall - l):.4f}, {min(1, recall + l):.4f})")


In [60]:
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy.

    Parameters:
    y_true : list or numpy array
        True labels.
    y_pred : list or numpy array
        Predicted labels.

    Returns:
    accuracy : float
        Accuracy of the model.
    n_samples : int
        Number of total samples.
    """
    correct_predictions = sum(y_true == y_pred)
    total_samples = len(y_true)
    
    accuracy = correct_predictions / total_samples if total_samples > 0 else 0
    return accuracy, total_samples

# Example Usage
accuracy, n_samples = compute_accuracy(y_true, y_pred)
l = compute_hoeffding_bound(n_samples)

print(f"Accuracy: {accuracy:.4f}")
print(f"95% Confidence Interval: ({max(0, accuracy - l):.4f}, {min(1, accuracy + l):.4f})")


NameError: name 'y_true' is not defined

In [ ]:
def vc_dimension_effect(n_samples, vc_dim, confidence=0.95):
    """
    Computes the theoretical sample size required to reduce the Hoeffding bound.

    Parameters:
    n_samples : int
        The available sample size.
    vc_dim : int
        The VC-dimension of the classifier.
    confidence : float
        Confidence level (default 95%).

    Returns:
    required_samples : int
        The estimated number of samples needed for a reduced confidence interval.
    """
    delta = 1 - confidence
    epsilon = np.sqrt(np.log(2 / delta) / (2 * n_samples))
    
    # Theoretical number of samples needed based on VC-dimension
    required_samples = (1 / epsilon**2) * (vc_dim * np.log(2 * n_samples / vc_dim) + np.log(2 / delta))
    
    return int(required_samples)

# Example Usage
vc_dim = 3
required_samples = vc_dimension_effect(n_samples, vc_dim)

print(f"Required samples for VC-dimension {vc_dim} to reduce interval: {required_samples}")
print(f"Using all data would likely result in a smaller confidence interval for accuracy.")
